In [17]:
import json, boto3, logging, time, asyncio
from boto3.dynamodb.conditions import Key
from botocore.exceptions import ClientError
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from farmhash import FarmHash32 as fhash

In [2]:
with open("../.KEYS/SECOND_PREPROCESSING_KEY.json") as f:
    aws_key = json.load(f)

# data path 정보 get
with open("../.KEYS/DATA_SRC_INFO.json", "r") as f:
    data_src = json.load(f)

# GEMINI API key 값 get
with open("../.KEYS/GEMINI_API_KEY.json", "r") as f:
    gemini_api_key = json.load(f)

# prompt data get
with open("../.DATA/PROMPT_INFO.json") as f:
    prompt_metadata = json.load(f)

In [3]:
import boto3

# DynamoDB 리소스 생성
dynamodb = boto3.client(
    'dynamodb',
    aws_access_key_id=aws_key['aws_access_key_id'],
    aws_secret_access_key=aws_key['aws_secret_key'],
    region_name=aws_key['region']
)

# 테이블 이름
dynamo_table_name = data_src['restore_table_name']

# 읽기 용량을 늘리기 위한 업데이트
response = dynamodb.update_table(
    TableName=dynamo_table_name,
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,  # 원하는 읽기 용량 단위
        'WriteCapacityUnits': 5   # 원하는 쓰기 용량 단위 (필요에 따라 설정)
    }
)

print("Table update response:", response)


Table update response: {'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'get_date', 'AttributeType': 'N'}, {'AttributeName': 'id', 'AttributeType': 'N'}], 'TableName': 'merged-data-table', 'KeySchema': [{'AttributeName': 'id', 'KeyType': 'HASH'}, {'AttributeName': 'get_date', 'KeyType': 'RANGE'}], 'TableStatus': 'UPDATING', 'CreationDateTime': datetime.datetime(2024, 8, 22, 16, 36, 22, 876000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(2024, 9, 3, 15, 41, 34, 718000, tzinfo=tzlocal()), 'LastDecreaseDateTime': datetime.datetime(2024, 9, 3, 15, 35, 12, 530000, tzinfo=tzlocal()), 'NumberOfDecreasesToday': 8, 'ReadCapacityUnits': 10, 'WriteCapacityUnits': 1}, 'TableSizeBytes': 16835415, 'ItemCount': 8400, 'TableArn': 'arn:aws:dynamodb:ap-northeast-2:533267279103:table/merged-data-table', 'TableId': '2c719fa9-8d09-4ecb-be34-b09f6f2674d5', 'TableClassSummary': {'TableClass': 'STANDARD'}, 'DeletionProtectionEnabled': False}, 'Respons

In [ ]:
# 데이터 스캔 함수
def scan_table(table_name, limit=200):
    scan_kwargs = {
        'TableName': table_name,
        'Limit': limit  # 페이지 크기 설정 (여기서는 200)
    }
    
    source_data = []
    
    while True:
        try:
            response = dynamodb.scan(**scan_kwargs)
            source_data.extend(response.get('Items', []))
            
            # 스캔된 데이터가 limit 수보다 작거나 'LastEvaluatedKey'가 없으면 종료
            if len(source_data) >= limit or 'LastEvaluatedKey' not in response:
                break
            
            # 계속해서 다음 페이지의 데이터를 가져오기 위한 키 설정
            scan_kwargs['ExclusiveStartKey'] = response['LastEvaluatedKey']
        
        except ClientError as e:
            print(f"An error occurred: {e}")
            break
    
    return source_data[:limit]  # 정확히 200개 항목만 반환

# 데이터 가져오기
items = scan_table(dynamo_table_name)

# 결과 출력
# print(f"Retrieved {len(items)} items.")
# for item in items:
#     print(item)


In [7]:
items[0]

{'company_id': {'N': '1023'},
 'site_symbol': {'S': 'PRO'},
 'job_title': {'S': '프로그램 개발 엔지니어(PM/PL)'},
 'job_category': {'S': '서버/백엔드, 프론트엔드'},
 'job_prefer': {'S': '구매 관련 프로젝트 수행 경력 보유자 엠로 개발프레임워크 및 유사 SCM 솔루션 개발 경험자 클라우드 플랫폼(AWS, Azure, Google Cloud) 경험 우대 프로젝트 관리 자격증(PMP) 보유자'},
 'crawl_url': {'S': 'https://career.programmers.co.kr/api/job_positions/8394'},
 'end_date': {'NULL': True},
 'status': {'S': 'released'},
 'post_status': {'BOOL': True},
 'required_career': {'BOOL': True},
 'job_id': {'N': '8394'},
 'crawl_domain': {'S': 'https://career.programmers.co.kr/'},
 'company_name': {'S': '엠로'},
 'resume_required': {'BOOL': True},
 'start_date': {'S': '2024-07-03'},
 'get_date': {'N': '20240828'},
 'job_tasks': {'S': '공급망관리(SCM) 솔루션 설계 및 개발 구매업무 프로세스 최적화 및 시스템 구축 프로젝트 수행 고객 요구사항 분석 및 솔루션 제안'},
 'id': {'N': '2902426399'},
 'stacks': {'S': 'Java, JavaScript, Spring, Oracle, MSSQL(Microsoft SQL Server)'},
 'job_requirements': {'S': 'IT 프로젝트 수행 경력 6년 이상 (PL 수행경험 2년 이상 포함) 웹 프로그래밍 능숙자 

In [9]:
wanted_data = [obj for obj in items if obj.get('site_symbol', '').get('S', '').upper() == "WAN"]
jobkorea_data = [obj for obj in items if obj.get('site_symbol', '').get('S', '').upper() == "JK"]
rocketpunch_data = [obj for obj in items if obj.get('site_symbol', '').get('S', '').upper() == "RP"]
programmers_data = [obj for obj in items if obj.get('site_symbol', '').get('S', '').upper() == "PRO"]

In [15]:
genai.configure(api_key=gemini_api_key['GEMINI_API'])

# Create the model
generation_config = {
  "temperature": 0.7,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  safety_settings = {
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
  }
)

chat_session = model.start_chat(
  history=[
  ]
)

In [16]:
def return_object_prompt(data, symbol_key):
    return data.get(symbol_key, {}).get("prompt")

def return_object_source_keys(data, symbol_key):
    return data.get(symbol_key, {}).get("source_key")

def return_concat_data_record(obj, data_dict):
    data_record = {
        "pid": obj.get("id"),
        "get_date": obj.get("get_date"),
        "site_symbol": obj.get("site_symbol"),
        "job_title": obj.get("job_title", None),
        "dev_stack": data_dict.get("dev_stack", []),
        "job_requirements": data_dict.get("job_requirements", []),
        "job_prefer": data_dict.get("job_prefer", []),
        "job_category": data_dict.get("job_category", []),
        "indurstry_type": data_dict.get("indurstry_type", []),
        "required_career": obj.get("required_career", None),
        "resume_required": obj.get("resume_required", None),
        "post_status": obj.get("post_status", None),
        "company_name": obj.get("company_name", None),
        "cid": fhash(obj.get("site_symbol")+obj.get("company_name")+str(int(obj.get("company_id")))),
        "start_date": obj.get("start_date", None),
        "end_date": obj.get("end_date", None),
        "crawl_domain": obj.get("crawl_domain", None),
        "crawl_url": obj.get("crawl_url", None)
    }
    
    return data_record

def upload_data(_item):
    # DynamoDB 클라이언트 생성
    dynamodb = boto3.resource(
        'dynamodb',
        aws_access_key_id=key['aws_access_key_id'],
        aws_secret_access_key=key['aws_secret_key'],
        region_name=key['region']
    )
    table = dynamodb.Table("precessed-data-table")
    table.put_item(Item=_item)

In [ ]:
prompt_data = prompt_metadata.get("data", {})
for _obj in source_data:
    _symbol = _obj.get('site_symbol', "").upper()
    if _symbol in prompt_data.keys():
        _data_source_keys = return_object_source_keys(prompt_data, _symbol)
        _prompt = return_object_prompt(prompt_data, _symbol).format(
            data_source_keys=_data_source_keys, 
            input_data=str(_obj)
            )
        try:
            response = chat_session.send_message(_prompt)
            json_data = response.text.replace("```json\n", "").replace("\n```", "")
            dict_data = json.loads(json_data)
            data_item = return_concat_data_record(obj=_obj, data_dict=dict_data)
        except Exception as e:
            print(e)
